# PLR初期設定

PLRのアイテムをJupyter Notebookで処理するためには、本NotebookにしたがってPLR初期設定を実施する必要があります。

# PLRバイナリのインストール

本環境では、PLR CUIバイナリをPLRへのアクセスに使用します。これは公開されていないので、別に共有されたものを **このNotebookと同じディレクトリ** に配置してください。

In [1]:
!mkdir -p ~/.groovy
!ln -s /opt/plrfs/lib ~/.groovy/lib

# PLRアカウントの初期設定

以下のCellを1つ1つ実行する。

In [2]:
import pexpect

plrcommand = 'plr'
plrcui = pexpect.spawn(plrcommand + " storage new googleDrive")

以下のセルを実行すると、URLが表示されるので、ブラウザで開く。ブラウザでGoogle認証を行うと、http://localhost:40385/... というURLが表示されるので、これを入力欄に貼り付ける。(ブラウザ画面にはエラーが表示されるが問題ない)

In [3]:
plrcui.expect(r'\s+https://accounts.google.com/.*')
print(plrcui.after.decode('utf8').strip())

callback_url = input('このセルの出力に表示されるURLをクリックし、Googleアカウントで認証を行い、ブラウザに表示されたURLを貼り付けてください。')
!curl -v -X GET "{callback_url}"

https://accounts.google.com/o/oauth2/auth?access_type=offline&approval_prompt=force&client_id=1001437678350-inrbi6862etroo8kdhshvm24hkuguta6.apps.googleusercontent.com&redirect_uri=http://localhost:39279/Callback&response_type=code&scope=https://www.googleapis.com/auth/drive
このセルの出力に表示されるURLをクリックし、Googleアカウントで認証を行い、ブラウザに表示されたURLを貼り付けてください。http://localhost:39279/Callback?code=4/0AY0e-g4kMqq0HOUkJ3yBz4R_jBb7cNqKH8gMqndTMocbf_Xtv1g3jhaaIStmDf0ZxoUrkA&scope=https://www.googleapis.com/auth/drive
Note: Unnecessary use of -X or --request, GET is already inferred.
*   Trying 127.0.0.1:39279...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 39279 (#0)
> GET /Callback?code=4/0AY0e-g4kMqq0HOUkJ3yBz4R_jBb7cNqKH8gMqndTMocbf_Xtv1g3jhaaIStmDf0ZxoUrkA&scope=https://www.googleapis.com/auth/drive HTTP/1.1
> Host: localhost:39279
> User-Agent: curl/7.68.0
> Accept: */*
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< Date: Mon, 07 Jun 2021 22:21:00 GMT
< Transfer-encoding: chu

PLRのパスフレーズを入力する。Personary設定時に指定したパスワードを以下に指定する。

In [4]:
import re
from getpass import getpass

passphrase = getpass()
plrcui.expect('Enter passphrase:')
storage_id = [re.match(r'.*Storage created:\s*([0-9]+):.*', line).group(1) for line in plrcui.before.decode('utf8').splitlines() if re.match(r'.*Storage created:\s*([0-9]+):.*', line)][0]
plrcui.sendline(passphrase)
plrcui.expect('# ')

storage_id

········


'1'

エラーとならなければ、これで設定は終了。

In [5]:
plrcui.sendline('exit')
plrcui.expect('Exiting... done.')

0

得られた設定を設定ファイルに書き出す。

In [6]:
import os

with open(os.path.expanduser('~/.plrprofile'), 'w') as f:
    f.write('''export STORAGE_ID={storage_id}
export PASSPHRASE="{passphrase}"'''.format(**locals()))

PLRデータ取得サービスを起動する。

In [7]:
!supervisorctl -s http://localhost:9001 status

plrfs                            STOPPED   Not started
rabbitmq                         RUNNING   pid 21, uptime 0:01:10


In [8]:
import time

!supervisorctl -s http://localhost:9001 start plrfs
# 起動完了するまで1分待つ
time.sleep(60)

plrfs: started


In [9]:
# 問題が発生した場合は以下のコマンドをコメントアウトして調査する。
# !cat /tmp/supervisor-plrfs.log

# 動作確認

In [10]:
import asyncio
from plrfs.rpc_client import PLRFSClient

loop = asyncio.get_event_loop()

チャンネル一覧を取得する。

In [11]:
client = await PLRFSClient(loop).connect()

channels = await client.get_files([])
channels

[{'kind': 'folder', 'id': '20210218034432_8WEg', 'name': 'family'},
 {'kind': 'folder', 'id': '20210218034432_eepe', 'name': 'coworkers'},
 {'kind': 'folder', 'id': '20210218034432_rMn9', 'name': 'friends'},
 {'kind': 'folder', 'id': '20210302031112_3GCW', 'name': '共有テスト'},
 {'kind': 'folder', 'id': '20210302070238_7B7I', 'name': 'Garmin Connect活動記録'},
 {'kind': 'folder', 'id': '20210302070238_GD5k', 'name': 'Cloze Test学習記録'},
 {'kind': 'folder', 'id': '20210406135701_CwTF', 'name': 'cog-pds-log'}]

In [12]:
# Garmin Connect Channelがなければエラーとなります。無視してください
garmin_channel = [c for c in channels if c['name'] == 'cog-pds-log'][0]
garmin_channel

{'kind': 'folder', 'id': '20210406135701_CwTF', 'name': 'cog-pds-log'}

In [15]:
import json

timeline_item_data = []
timeline_items = await client.get_files([garmin_channel['id']])
for item in timeline_items[-20:]:
    timeline_properties = await client.get_files([garmin_channel['id'], item['id']])
    assert 'cnt' in [p['name'] for p in timeline_properties]
    assert 'begin' in [p['name'] for p in timeline_properties]
    assert 'cogPDSJSON' in [p['name'] for p in timeline_properties]
    summary = await client.get_file([garmin_channel['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'cnt'][0]])
    begin = await client.get_file([garmin_channel['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'begin'][0]])
    cogPDSJSON = await client.get_file([garmin_channel['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'cogPDSJSON'][0]])
    timeline_item_data.append((summary['content'].decode('utf8'), begin['content'].decode('utf8'), json.loads(cogPDSJSON['content'])))
#    timeline_item_data
timeline_item_data[-5:]

[{'kind': 'file', 'content': None, 'id': '#20210607071918_tki8#begin', 'name': 'begin'}, {'kind': 'file', 'content': None, 'id': '#20210607071918_tki8#cnt', 'name': 'cnt'}, {'kind': 'file', 'content': None, 'id': '#20210607071918_tki8#cogPDSJSON', 'name': 'cogPDSJSON'}, {'kind': 'file', 'content': None, 'id': '#20210607071918_tki8#cogPDSUser', 'name': 'cogPDSUser'}, {'kind': 'file', 'content': None, 'id': '#20210607071918_tki8#creator', 'name': 'creator'}]
[{'kind': 'file', 'content': None, 'id': '#20210607071913_7Kdx#begin', 'name': 'begin'}, {'kind': 'file', 'content': None, 'id': '#20210607071913_7Kdx#cnt', 'name': 'cnt'}, {'kind': 'file', 'content': None, 'id': '#20210607071913_7Kdx#cogPDSJSON', 'name': 'cogPDSJSON'}, {'kind': 'file', 'content': None, 'id': '#20210607071913_7Kdx#cogPDSUser', 'name': 'cogPDSUser'}, {'kind': 'file', 'content': None, 'id': '#20210607071913_7Kdx#creator', 'name': 'creator'}]
[{'kind': 'file', 'content': None, 'id': '#20210607071916_HekS#begin', 'name':

[('*動画視聴ログ記録: 動画を視聴しました: 再生時間 0 秒',
  '2021-06-07T16:05:16.593Z',
  {'meta': {'task': {'id': 'Sb7lnzVw86yQwLxmXFFA', 'title': '動画視聴ログ記録'},
    'started': 1623081915098,
    'recorded': 1623081916591,
    'finished': 1623081916591,
    'duration': 1493},
   'data': {'history': [{'time': 1623081916112,
      'event': {'type': 'StateChange', 'state': 'UNSTARTED'},
      'state': {'isMuted': False,
       'volume': 100,
       'playbackRate': 1,
       'currentTime': 0,
       'playerState': 'UNSTARTED',
       'playbackQuality': 'unknown'}},
     {'time': 1623081916113,
      'event': {'type': 'StateChange', 'state': 'CUED'},
      'state': {'isMuted': False,
       'volume': 100,
       'playbackRate': 1,
       'currentTime': 0,
       'playerState': 'CUED',
       'playbackQuality': 'unknown'}},
     {'time': 1623081916547,
      'event': {'type': 'StateChange', 'state': 'UNSTARTED'},
      'state': {'isMuted': False,
       'volume': 100,
       'playbackRate': 1,
       'currentTime'